In [ ]:
import torch
from torch import nn
import utils.data_utils as data_utils
from training.MADGAN_train import train_kdd99, train_financial
from models.MADGAN import Generator, Discriminator, AnomalyDetector
from utils import evaluation

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# Hyperparameters

In [ ]:
# choose dataset (kdd99, aapl, gm, axp)
dataset = "kdd99"

In [ ]:
num_features = 6
seq_len = 30
seq_stride = 10
gen_seq_len = 1

random_seed = 0
num_epochs = 100
batch_size = 256
lr = 1e-5
wd = 5e-7

latent_dim = 250
hidden_dim = 500
anomaly_threshold = 0.5

# Load data

In [ ]:
if dataset == "kdd99":
    train_dl, test_dl = data_utils.kdd99(seq_len, seq_stride, num_features, gen_seq_len, batch_size)
else:
    file_path = 'data/financial_data/Stocks/'+dataset+'.us.txt'
    tscv_dl_list = data_utils.load_stock_as_crossvalidated_timeseries(file_path, seq_len, seq_stride, gen_seq_len, batch_size, normalise=True)

# Model

In [ ]:
generator = Generator(input_dim=latent_dim,hidden_size=hidden_dim,output_dim=num_features).to(device=DEVICE)

In [ ]:
discriminator = Discriminator(input_dim=num_features,hidden_size=hidden_dim).to(device=DEVICE)

# Loss and Optimizer

In [ ]:
def loss_function(inputs, targets):
    return nn.BCELoss()(inputs, targets)

In [ ]:
discriminator_optim = torch.optim.Adam(discriminator.parameters(), lr=lr, weight_decay=wd)
generator_optim = torch.optim.Adam(generator.parameters(), lr=lr, weight_decay=wd)

# Train & Evaluate

In [ ]:
if dataset == "kdd99":
    train_kdd99(seq_len, latent_dim, train_dl, test_dl, discriminator, generator, discriminator_optim, generator_optim, anomaly_threshold, loss_function, random_seed, num_epochs, DEVICE)
else:
    train_financial(seq_len, latent_dim, tscv_dl_list, discriminator, generator, discriminator_optim, generator_optim, anomaly_threshold, loss_function, random_seed, num_epochs, DEVICE)